# Exemplo 2: Automotivo - Sensores IoT de Veículos

Este notebook demonstra o processamento distribuído de dados de sensores de veículos usando **Dask**.

**Cenário**: Analisar dados de uma frota de veículos (velocidade, RPM, temperatura do óleo) para identificar comportamentos de direção agressiva.

## 1. Configuração do Ambiente
Instalação do Dask.

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client

# Iniciar um cluster Dask local
client = Client()
print(client)

## 2. Simulador de Dados
Gerar grandes arquivos CSV simulando logs de telemetria de 1000 veículos.

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os

# Criar diretório para dados
os.makedirs('vehicle_data', exist_ok=True)

def generate_vehicle_logs(file_id):
    num_records = 10000
    df = pd.DataFrame({
        'vehicle_id': np.random.randint(1, 1001, num_records),
        'speed_kmh': np.random.normal(80, 20, num_records),
        'rpm': np.random.normal(3000, 1000, num_records),
        'oil_temp': np.random.normal(90, 10, num_records),
        'timestamp': pd.date_range(start='2024-01-01', periods=num_records, freq='s')
    })
    # Introduzir anomalias (direção agressiva)
    df.loc[df['speed_kmh'] > 140, 'aggressive_flag'] = 1
    df.loc[df['speed_kmh'] <= 140, 'aggressive_flag'] = 0
    
    df.to_csv(f'vehicle_data/log_{file_id}.csv', index=False)

# Gerar 20 arquivos CSV em paralelo é desnecessário aqui, vamos gerar serialmente pois é rápido
print("Gerando dados simulados...")
for i in range(20):
    generate_vehicle_logs(i)
print("Dados gerados em 'vehicle_data/'")

## 3. Processamento Distribuído com Dask
Ler todos os arquivos CSV como um único Dask DataFrame e calcular estatísticas.

In [ ]:
# Ler múltiplos CSVs (Lazy Evaluation)
ddf = dd.read_csv('vehicle_data/log_*.csv')

# Visualizar primeiras linhas
ddf.head()

In [ ]:
# Transformação: Calcular média de velocidade por veículo
avg_speed = ddf.groupby('vehicle_id')['speed_kmh'].mean()

# Filtro: Veículos com média de velocidade muito alta
fast_drivers = avg_speed[avg_speed > 110]

# Ação: Computar resultados (aqui o Dask distribui o trabalho)
result = fast_drivers.compute()

print(f"Veículos rápidos detectados: {len(result)}")
print(result.head())

In [ ]:
# Análise mais complexa: Correlacionar RPM com Temperatura do Óleo
correlation = ddf[['rpm', 'oil_temp']].corr().compute()
print("Matriz de Correlação:")
print(correlation)

In [ ]:
# Limpeza
client.close()